In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,40749
2,Huelva,Confirmados PDIA 14 días,3231
3,Huelva,Tasa PDIA 14 días,"629,615916752733"
4,Huelva,Confirmados PDIA 7 días,1452
5,Huelva,Tasa PDIA 7 dias,"282,94717150262096"
6,Huelva,Total Confirmados,40959
7,Huelva,Curados,34843
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  40749.0


/tmp/ipykernel_3286/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11341.0


/tmp/ipykernel_3286/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3286/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3286/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3286/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 431 personas en los últimos 7 días 

Un positivo PCR cada 166 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,40749.0,1452.0,3231.0,513170.0,282.947172,629.615917,668.0
Huelva-Costa,23909.0,803.0,1860.0,289548.0,277.328802,642.380538,368.0
Condado-Campiña,12831.0,526.0,1032.0,156231.0,336.680940,660.560324,257.0
Huelva (capital),11341.0,334.0,866.0,143837.0,232.207290,602.070399,169.0
Lepe,2679.0,114.0,220.0,27880.0,408.895265,789.096126,55.0
Moguer,1773.0,84.0,175.0,21867.0,384.140486,800.292678,53.0
Sierra de Huelva-Andévalo Central,3627.0,119.0,332.0,67391.0,176.581443,492.647386,41.0
Ayamonte,1739.0,79.0,147.0,21104.0,374.336619,696.550417,37.0
Isla Cristina,2753.0,77.0,196.0,21393.0,359.930818,916.187538,33.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Sanlúcar de Guadiana,23.0,14.0,16.0,403.0,3473.945409,3970.223325,9.0
Cumbres Mayores,117.0,15.0,49.0,1749.0,857.632933,2801.600915,4.0
Corteconcepción,38.0,4.0,13.0,536.0,746.268657,2425.373134,2.0
Marines (Los),14.0,7.0,8.0,399.0,1754.385965,2005.012531,5.0
Encinasola,160.0,2.0,23.0,1310.0,152.671756,1755.725191,1.0
Nava (La),6.0,3.0,4.0,258.0,1162.790698,1550.387597,1.0
Cañaveral de León,13.0,3.0,6.0,396.0,757.575758,1515.151515,2.0
Escacena del Campo,194.0,23.0,33.0,2287.0,1005.684303,1442.938347,10.0
Villablanca,243.0,20.0,34.0,2885.0,693.240901,1178.509532,7.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Encinasola,160.0,2.0,23.0,1310.0,152.671756,1755.725191,1.0,0.086957
Almonaster la Real,85.0,1.0,10.0,1788.0,55.928412,559.284116,0.0,0.100000
Jabugo,83.0,1.0,7.0,2260.0,44.247788,309.734513,0.0,0.142857
Puebla de Guzmán,163.0,2.0,11.0,3092.0,64.683053,355.756792,2.0,0.181818
Santa Ana la Real,21.0,1.0,5.0,475.0,210.526316,1052.631579,0.0,0.200000
Cerro de Andévalo (El),184.0,2.0,10.0,2327.0,85.947572,429.737860,1.0,0.200000
Aracena,618.0,8.0,29.0,8255.0,96.910963,351.302241,1.0,0.275862
Cortegana,266.0,7.0,23.0,4602.0,152.107779,499.782703,4.0,0.304348
Cumbres Mayores,117.0,15.0,49.0,1749.0,857.632933,2801.600915,4.0,0.306122
